# Bloom基于LoRA微调实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import json


# https://pytorch.org/
# pip install lib/bitsandbytes-0.40.1.post1-py3-none-win_amd64.whl

## Step2 加载数据集

In [2]:
def load_instruction(instruct_dir):
    input_data = []
    with open(instruct_dir, "r",encoding="utf-8") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            d = json.loads(line)
            input_data.append(d)
    return input_data
ds = load_instruction("./data/llama_data.json")

In [3]:
ds[:3]

[{'instruction': '一名年龄在70岁的女性，出现了晕厥、不自主颤抖、情绪不稳等症状，请详细说明其手术治疗和术前准备。',
  'input': '',
  'output': '该病需要进行电极导线、脉冲发生器和永久心脏起搏器置入术，并需要使用镇静药物和局麻对病人进行手术治疗。术前准备包括1-3天的时间进行术前检查和生活方式的调整。'},
 {'instruction': '一个患者的卵巢小细胞癌转移至其它部位，是否有必要进行手术治疗？',
  'input': '',
  'output': '当卵巢小细胞癌转移至其它部位时，手术治疗的效果可能不理想，因此一般不推荐进行手术治疗。针对转移病灶，可以采用化疗、放疗等治疗手段进行综合治疗。'},
 {'instruction': '一位60岁男性患者由于肝动脉瘤出现肝功能异常，具体表现为黄疸，该如何诊治？',
  'input': '',
  'output': '该患者可能出现了肝动脉瘤并发症之一的梗阻性黄疸，需要进行腹部平片、彩色多普勒超声、CTA、动脉造影等检查后确定诊断，治疗方面应按照病因治疗，如若为肝动脉炎、动脉粥样硬化等病因，应该采取手术治疗，而若是单纯的肝动脉瘤，可以进行介入治疗或手术治疗。'}]

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("model/langboat/bloom-800m-zh")

tokenizer

BloomTokenizerFast(name_or_path='model/langboat/bloom-800m-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
tokenizer.padding_side = "right"

In [6]:
tokenizer.pad_token_id = 2

In [7]:
def process_func(example):
    MAX_LENGTH = 400
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=BloomTokenizerFast(name_or_path='model/langboat/bloom-800m-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [9]:
# tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
# tokenized_ds
tokenized_ds = []
for row in ds:
    tokenized_ds.append(process_func(row))

In [10]:
print(tokenized_ds[0]["input_ids"])

[26283, 29, 7595, 1228, 12264, 587, 2342, 16599, 6883, 355, 17803, 20474, 19255, 554, 643, 17430, 37788, 24406, 554, 18871, 643, 7058, 42254, 355, 2680, 11518, 6428, 936, 13193, 6623, 642, 2629, 1230, 7256, 672, 189, 4340, 17245, 29, 210, 1811, 2453, 43817, 1936, 3889, 2728, 2665, 554, 41385, 4766, 33201, 14095, 19477, 1533, 23431, 2339, 2884, 1359, 2629, 355, 1437, 2732, 2527, 4233, 7963, 10847, 642, 2460, 6125, 1079, 15003, 2557, 13193, 6623, 420, 2629, 1230, 7256, 2542, 33456, 1317, 11408, 2557, 2629, 1230, 8116, 642, 34118, 373, 9734, 420, 2]


In [11]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

'当卵巢小细胞癌转移至其它部位时，手术治疗的效果可能不理想，因此一般不推荐进行手术治疗。针对转移病灶，可以采用化疗、放疗等治疗手段进行综合治疗。</s>'

## Step4 创建模型

In [12]:
import torch

print(torch.cuda.is_available())

model = AutoModelForCausalLM.from_pretrained("model/langboat/bloom-800m-zh", 
                                             low_cpu_mem_usage=True,
                                             torch_dtype=torch.half,
                                             device_map="cuda")


True


In [13]:
model.dtype

torch.float16

## Lora

### PEFT Step1 配置文件

In [14]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
                task_type=TaskType.CAUSAL_LM,
                r=8,
                target_modules=['query_key_value','dense_4h_to_h']
                )
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'dense_4h_to_h', 'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

### PEFT Step2 创建模型

In [15]:
model = get_peft_model(model, config)

In [16]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='model/langboat/bloom-800m-zh', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'dense_4h_to_h', 'query_key_value'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [17]:
model.print_trainable_parameters()

trainable params: 2,654,208 || all params: 753,495,552 || trainable%: 0.3522526434237


In [18]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

base_model.model.transformer.word_embeddings.weight torch.Size([46145, 1536]) torch.float16
base_model.model.transformer.word_embeddings_layernorm.weight torch.Size([1536]) torch.float16
base_model.model.transformer.word_embeddings_layernorm.bias torch.Size([1536]) torch.float16
base_model.model.transformer.h.0.input_layernorm.weight torch.Size([1536]) torch.float16
base_model.model.transformer.h.0.input_layernorm.bias torch.Size([1536]) torch.float16
base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.weight torch.Size([4608, 1536]) torch.float16
base_model.model.transformer.h.0.self_attention.query_key_value.base_layer.bias torch.Size([4608]) torch.float16
base_model.model.transformer.h.0.self_attention.query_key_value.lora_A.default.weight torch.Size([8, 1536]) torch.float16
base_model.model.transformer.h.0.self_attention.query_key_value.lora_B.default.weight torch.Size([4608, 8]) torch.float16
base_model.model.transformer.h.0.self_attention.dense.weight torch

## Step5 配置训练参数

In [21]:
##   adam_epsilon=1e-4
args = TrainingArguments(
    output_dir="bloomoutput",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=100,
    num_train_epochs=6,
    adam_epsilon=1e-4
)

# adam_epsilon   1e-8  0.000000001

## Step6 创建训练器

In [22]:
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=tokenized_ds,
#     data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
# )


trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_ds,
    eval_dataset=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

## Step7 模型训练

In [23]:
trainer.train()

wandb: Currently logged in as: dafei1288. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,2.327500
200,2.155600
300,2.139900
400,2.121500
500,2.105900
600,2.032800
700,2.040100
800,2.041400
900,2.028600
1000,2.011600


D:\working\mycode\bloom_med_lora\venv\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in model/langboat/bloom-800m-zh - will assume that the vocabulary was not modified.
  warnings.warn(
D:\working\mycode\bloom_med_lora\venv\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in model/langboat/bloom-800m-zh - will assume that the vocabulary was not modified.
  warnings.warn(
D:\working\mycode\bloom_med_lora\venv\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in model/langboat/bloom-800m-zh - will assume that the vocabulary was not modified.
  warnings.warn(
D:\working\mycode\bloom_med_lora\venv\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in model/langboat/bloom-800m-zh - will assume that the vocabulary was not modified.
  warnings.warn(
D:\working\mycode\bloom_med_lora\venv\Lib\site-packages\peft\utils\save_

TrainOutput(global_step=6492, training_loss=1.7225325720147153, metrics={'train_runtime': 2595.0738, 'train_samples_per_second': 20.018, 'train_steps_per_second': 2.502, 'total_flos': 1.5789185353322496e+16, 'train_loss': 1.7225325720147153, 'epoch': 5.998613998613998})

## Step8 模型推理

In [24]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer ,truncation=True)

ipt = "Human: {}\n{}".format("关节部位红肿疼痛，排尿困难,怎么办？", "").strip() + "\n\nAssistant: "
print(pipe(ipt, max_length=400, do_sample=True, ))

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFo

[{'generated_text': 'Human: 关节部位红肿疼痛，排尿困难,怎么办？\n\nAssistant: 这些症状可能是其他疾病引起的，建议去内科进行详细的检查和治疗。'}]


## Step9 合并权重

In [3]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

base_model_dir = 'model/langboat/bloom-800m-zh'
lora_model_dir = 'bloomoutput/checkpoint-6000'

tokenizer = AutoTokenizer.from_pretrained(base_model_dir,trust_remote_code=True)

model = AutoPeftModelForCausalLM.from_pretrained(lora_model_dir, device_map='cuda', torch_dtype=torch.bfloat16)

model = model.merge_and_unload()
output_dir = 'bloomoutput/bloom-800m-zh-merged-med'
model.save_pretrained(output_dir)
torch.save(model.state_dict(), 'bloomoutput/bloom-800m-zh-merged-med/pytorch_model.bin'.format(6000))

# Step10 推理

In [5]:
from transformers import pipeline,AutoTokenizer,AutoModelForCausalLM
from transformers import pipeline
retokenizer = AutoTokenizer.from_pretrained("model/langboat/bloom-800m-zh")
remodel = AutoModelForCausalLM.from_pretrained("bloomoutput/bloom-800m-zh-merged-med", 
                                             low_cpu_mem_usage=True,
                                             torch_dtype=torch.half,
                                             device_map="cuda")
repipe = pipeline("text-generation", model=remodel, tokenizer=retokenizer ,truncation=True)

ipt = "Human: {}\n{}".format("关节部位红肿疼痛，排尿困难,怎么办？", "").strip() + "\n\nAssistant: "
print(repipe(ipt, max_length=400, do_sample=True, ))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Human: 关节部位红肿疼痛，排尿困难,怎么办？\n\nAssistant: 需要就诊，医生会综合病情制定治疗方案，包括药物治疗和手术治疗。药物治疗包括前列地尔、普鲁卡因等。'}]
